# WNTR Pipe Break Tutorial
The following tutorial covers how to run a simple pipe break analysis using WNTR.

## Imports
Import WNTR and additional Python packages that are needed for the tutorial.
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Pandas is used for data manipulation and analysis
- Matplotlib is used to create graphics

In [ ]:
import wntr
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Water network model
If adapting code for a different EPANET input (INP) file, the correct file path and desired simulation parameters will need to be updated.

In [ ]:
# Identify file path to inp file
inp_file = 'networks/Net3.inp'

# Create water network model 
wn = wntr.network.WaterNetworkModel(inp_file)

# Calculate population per junction
population = wntr.metrics.population(wn)

## Pipe break parameters
The parameters `minimum_pressure` and `required_pressure` are used for pressure dependent demand (PDD) simulations. Nodes with pressures below minimum pressure will not receive any water, and node pressures need to be at least the required pressure to receive all of the requested demand.

The parameter `min_pipe_diam` defines the lower limit of pipe diameters to include in analysis.  

In [ ]:
# Define simulation parameters 
start_time = 2*3600 # 2 hours
break_duration = 12*3600 # 12 hours
total_duration = start_time + break_duration # 14 hours

minimum_pressure = 3.52 # 5 psi
required_pressure = 14.06 # 20 psi 

min_pipe_diam = 0.3048 # 12 inch

## Baseline simulation
The baseline simulation can be used to identify non-zero (NZD) junctions that fall below minimum pressure during normal operating conditions. This step helps determine which junctions that experience low pressures during the disaster simulation are a direct result of the disaster and not normal operating conditions. 

In [ ]:
# Calculate the average expected demand (AED) and identify the junctions with non-zero AED
AED = wntr.metrics.average_expected_demand(wn)
nzd_junct = AED[AED > 0].index

# Set hydraulic parameters
wn.options.hydraulic.demand_model = 'PDD'    
wn.options.time.duration = total_duration
wn.options.hydraulic.minimum_pressure = minimum_pressure
wn.options.hydraulic.required_pressure = required_pressure 

# Simulate the hydraulics
sim = wntr.sim.WNTRSimulator(wn)
results = sim.run_sim()

# Save junction pressure results and identify junctions that fall below minimum pressure
pressure = results.node['pressure'].loc[start_time::, nzd_junct]
normal_pressure_below_pmin = pressure.columns[(pressure < minimum_pressure).any()]

## Pipe break simulations
A try/except/finally approach is taken to ensure the script can finish running and still catch any convergence issues that might have occurred due to a pipe break. A user can revisit nodes with failed simulations individually to determine the cause of failure, if desired. 

In [ ]:
# Create dictionary to save results
analysis_results = {} 

# Query all pipes with diameter greater than threshold set earlier to include in analysis
pipes_of_interest = wn.query_link_attribute('diameter', np.greater_equal, min_pipe_diam)

# Simulate pipe break (simulated as pipe closures) for each pipe
for pipe_name in pipes_of_interest.index: 
    wn = wntr.network.WaterNetworkModel(inp_file)
    wn.options.hydraulic.demand_model = 'PDD'    
    wn.options.time.duration = total_duration
    wn.options.hydraulic.minimum_pressure = minimum_pressure
    wn.options.hydraulic.required_pressure = required_pressure
    
    # Create pipe closure control and apply to pipe of interest
    pipe = wn.get_link(pipe_name)
    act = wntr.network.controls.ControlAction(pipe, 'status', 0)
    cond = wntr.network.controls.SimTimeCondition(wn, 'Above', start_time)
    ctrl = wntr.network.controls.Control(cond, act)
    wn.add_control('close pipe ' + pipe_name, ctrl)
    
    try:
        # Simulate hydraulics
        sim = wntr.sim.WNTRSimulator(wn) 
        sim_results = sim.run_sim()
 
        # Identify impacted junctions using pressure results
        sim_pressure = sim_results.node['pressure'].loc[start_time::, nzd_junct]
        sim_pressure_below_pmin = sim_pressure.columns[(sim_pressure < minimum_pressure).any()]
        impacted_junctions = set(sim_pressure_below_pmin) - set(normal_pressure_below_pmin)
        impacted_junctions = list(impacted_junctions)
        
    except Exception as e:
        # Identify failed simulations and the reason
        impacted_junctions = None
        print(pipe_name, ' Failed:', e)

    finally:
        # Save simulation results
        analysis_results[pipe_name] = impacted_junctions

## Results

In [ ]:
# Calculate and save junction and population impact results to dictionary
num_junctions_impacted = {}
num_people_impacted = {}
for pipe_name, impacted_junctions in analysis_results.items():
    if impacted_junctions is not None:
        num_junctions_impacted[pipe_name] = len(impacted_junctions)
        num_people_impacted[pipe_name] = population[impacted_junctions].sum()

In [ ]:
# Set colormap for network maps
cmap=matplotlib.colormaps['viridis']

# Plot junctions impacted due to pipe breaks
# The parameter `link_range` can be adjusted to better suit the simulation results of the network used in the analysis  
wntr.graphics.plot_network(wn, link_attribute=num_junctions_impacted, node_size=0, link_width=2, 
                           link_range=[0,10], link_cmap = cmap, link_colorbar_label='Junctions Impacted', 
                           title='Number of junctions impacted by each pipe closure')

# Plot population impacted due to pipe breaks
wntr.graphics.plot_network(wn, link_attribute=num_people_impacted, node_size=0, link_width=2, 
                           link_range=[0,5000], link_cmap = cmap, link_colorbar_label='Population Impacted',
                           title='Number of people impacted by each pipe closure')

## Save results to CSV files

In [ ]:
# Save the junction impacted results to CSV
# Check to verify the file was created in the directory
num_junctions_impacted = pd.Series(num_junctions_impacted)
num_junctions_impacted.to_csv('pipe_break_junctions_impacted.csv')

# Save the population impacted results to CSV
num_people_impacted = pd.Series(num_people_impacted)
num_people_impacted.to_csv('pipe_break_people_impacted.csv')